# Agent State Machine

**ARRANGE:** index the document

In [2]:
# Index test doc
import os
os.environ["LOG_LEVEL"]="DEBUG"

from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
    "type": "rag",
    "url": "http://gai-rag-svr:12036/gen/v1/rag",
    "ws_url": "ws://gai-rag-svr:12036/gen/v1/rag/index-file/ws"
})
cwd = os.getcwd()
await rag.index_document_async(collection_name="demo", file_path=f"{cwd}/where_to_find_the_best_chicken_rice_in_singapore.txt")

DEBUG    httppost:url=http://gai-rag-svr:12036/gen/v1/rag/index-file
DEBUG    httppost:data=None
DEBUG    httppost:files={'file': ('where_to_find_the_best_chicken_rice_in_singapore.txt',
          <_io.BufferedReader name='/workspaces/gai/gai-persona/tests/integration_test/3_fsm/where_to_find_the_best_chicken_rice_in_singapore.txt'>,
          'application/pdf'),
 'req': (None,
         '{"Id":null,"CollectionName":"demo","FilePath":"/workspaces/gai/gai-persona/tests/integration_test/3_fsm/where_to_find_the_best_chicken_rice_in_singapore.txt","FileType":"","Source":"","ByteSize":null,"Title":"","Abstract":null,"Authors":"","Publisher":"","PublishedDate":"","Comments":"","Keywords":"","ChunkGroupId":null,"ChunkSize":null,"ChunkOverlap":null,"ChunkCount":null}',
         'application/json')}
ERROR    index_document_async: Error indexing file. error=Connection Error. Is the service Running?


Exception: Connection Error. Is the service Running?

## State 0. Init

The starting point is to create the string representation of the following mermaid diagram as follows:


In [1]:
state_diagram = """stateDiagram-v2
    INIT --> TOOL_CHOICE: next / on_TOOL_CHOICE

    TOOL_CHOICE --> CRAFT_TEXT_PROMPT: next / on_CRAFT_PROMPT / use_text
        CRAFT_TEXT_PROMPT --> GENERATE: next / on_GENERATE
        GENERATE --> PROCESS: next / on_PROCESS / use_text
        PROCESS --> END : next

    TOOL_CHOICE --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT / use_google
        TOOL_CALL --> GOOGLE: next / on_GOOGLE / use_google
        GOOGLE --> GENERATE: next / on_GENERATE

    TOOL_CHOICE --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT / use_retrieval
        TOOL_CALL --> RETRIEVAL: next / on_RETRIEVAL / use_retrieval
        RETRIEVAL --> GENERATE: next / on_GENERATE

    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL
    """

```mermaid
stateDiagram-v2
    INIT --> TOOL_CHOICE: next / on_TOOL_CHOICE

    TOOL_CHOICE --> CRAFT_TEXT_PROMPT: next / on_CRAFT_PROMPT / use_text
        CRAFT_TEXT_PROMPT --> GENERATE: next / on_GENERATE
        GENERATE --> PROCESS: next / on_PROCESS / use_text
        PROCESS --> END : next

    TOOL_CHOICE --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT / use_google
        TOOL_CALL --> GOOGLE: next / on_GOOGLE / use_google
        GOOGLE --> GENERATE: next / on_GENERATE

    TOOL_CHOICE --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT / use_retrieval
        TOOL_CALL --> RETRIEVAL: next / on_RETRIEVAL / use_retrieval
        RETRIEVAL --> GENERATE: next / on_GENERATE

    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL
```

In [2]:
# ARRANGE
import os
os.environ["LOG_LEVEL"]="WARNING"
from gai.lib.common.notebook import highlight,print_colored
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__name__), '..')))
from TestAgentHelper import TestAgentHelper
agent = TestAgentHelper.CreateAgent("Which famous chicken rice in Singapore also sells char siew?",state_diagram=state_diagram)


Machine validation successful.


## State 1. TOOL_CHOICE

The first state is managed by `use_TOOL_CHOICE_handler` state handler function.
The agent should decide to use "retrieval".

In [ ]:
# ACT
agent.next()

# ASSERT
TestAgentHelper.ShowAgent(agent)

## State 2. CRAFT_TOOL_PROMPT

CRAFT_TOOL_PROMPT is triggered because the agent recommends "retrieval". System message should return "".

In [4]:
# ACT
agent.next()

# ASSERT
TestAgentHelper.ShowAgent(agent)

step=2
state=CRAFT_TOOL_PROMPT
tool_choice=required
tool_name='retrieval




MONOLOGUE



Alfred, Which famous chicken rice in Singapore also sells char siew?





## State 3. TOOL_CALL


In [6]:
# ACT
agent.next()
print(agent.content)

# ASSERT
TestAgentHelper.ShowAgent(agent)

DEBUG    httppost:url=http://localhost:12031/gen/v1/chat/completions
DEBUG    httppost:data={'json_schema': None,
 'max_new_tokens': 1000,
 'max_tokens': 2000,
 'messages': [{'content': '', 'role': 'system'},
              {'content': 'Alfred, Which famous chicken rice in Singapore also '
                          'sells char siew?',
               'role': 'user'},
              {'content': '', 'role': 'assistant'}],
 'stop_conditions': None,
 'stream': False,
 'temperature': 1e-08,
 'tool_choice': 'required',
 'tools': ['function',
           {'arguments': {'properties': {'search_query': {'description': 'The '
                                                                         'most '
                                                                         'effective '
                                                                         'search '
                                                                         'query '
                                                 

{"type": "function", "name": "retrieval", "arguments": "{\"search_query\": \"famous chicken rice in singapore also sells char siew\"}"}
step=3
state=TOOL_CALL
tool_choice=required
tool_name='retrieval




MONOLOGUE



Alfred, Which famous chicken rice in Singapore also sells char siew?





## State 4. RETRIEVAL



In [7]:
# ACT
agent.next()

# ASSERT
TestAgentHelper.ShowAgent(agent)

DEBUG    httppost:url=http://localhost:12036/gen/v1/rag/retrieve
DEBUG    httppost:data={'collection_name': 'demo',
 'n_results': 4,
 'query_texts': 'Which famous chicken rice in Singapore also sells char siew?'}
DEBUG    httppost:files=None


step=4
state=RETRIEVAL
tool_choice=none
tool_name='text




MONOLOGUE



👩‍🔬, use only the information provided to you by the user to answer the user''s question. 
            If the information is insufficient for 👩‍🔬 to derive an answer, just say ''I cannot find relevant information in my document store to answer the question correctly.'' 
            👩‍🔬 is to provide an in-depth analysis to the question based only on the information provided by the user and nothing more.
            👩‍🔬 will give a real-life example to support illustrating your point and contrasting it with a counter-example. 
            👩‍🔬 will also proofread and edit the content before responding. 
            👩‍🔬 will provide your own reasoned subjective perspective, noting where your view differs from or expands on the contents.
            Rules:
                - Consolidate the materials provided by the user and then organise them point by point.
                - Provide as much details as you can extract from the materials provided by the user.
                - Begin your re


            Refer to the following context: `["well as salt-baked chicken. With the diversity presented in a single dish, Singapore's chicken rice definitely draws in curious diners to uncover more of the Lion City's chicken rice culture. Being equally fascinated with Singapore's chicken rice, the MICHELIN Guide inspectors share with us where to get the best ones in Singapore, so read on below! Boon Tong Kee (Balestier Road) 399, 401, 403 Balestier Road, 329801 Singapore $", "15 September 2023 There are many dishes that are very much representative of Singapore, and one of the most iconic of them is, of course, chicken rice. Loosely referred to as Singapore's national dish, the ubiquitous Hainanese chicken rice requires a meticulous preparation process. To make this classic chicken rice dish, the chicken is first scalded until cooked through before it is dipped in ice-cold water to keep the meat soft and tender. Served with chicken broth, ginger sauce, black soy sauce, and chilli, the

## State 5. GENERATE



In [8]:
# ACT
agent.next()
for chunk in agent.streamer:
    print(chunk, end="", flush=True)

DEBUG    httppost:url=http://localhost:12031/gen/v1/chat/completions
DEBUG    httppost:data={'json_schema': None,
 'max_new_tokens': 1000,
 'max_tokens': 2000,
 'messages': [{'content': '👩\u200d🔬, use only the information provided to you '
                          "by the user to answer the user''s question. If the "
                          'information is insufficient for 👩\u200d🔬 to derive '
                          "an answer, just say ''I cannot find relevant "
                          'information in my document store to answer the '
                          "question correctly.'' 👩\u200d🔬 is to provide an "
                          'in-depth analysis to the question based only on the '
                          'information provided by the user and nothing more. '
                          '👩\u200d🔬 will give a real-life example to support '
                          'illustrating your point and contrasting it with a '
                          'counter-example. 👩\u200d🔬 w

 According to my document store, there is no information provided about any famous chicken rice in Singapore that also sells char siew. However, it is worth noting that the context does not mention any other dishes served by the restaurants mentioned. Therefore, it is possible that some of these restaurants may serve char siew as part of their menu, but this information is not provided in the context.

To answer your question more definitively, I would need additional information about which specific restaurants or chains of restaurants in Singapore are known for both chicken rice and char siew. Once I have that information, I can search my document store for more detailed information about those restaurants and their menus.

For example, let's say we know that the restaurant "Lau Pa Sat" is famous for both chicken rice and char siew. In that case, I could search my document store for more information about Lau Pa Sat and find out that they do indeed serve char siew as part of their me

In [9]:
# ASSERT
TestAgentHelper.ShowAgent(agent)


step=5
state=GENERATE
tool_choice=none
tool_name='text




MONOLOGUE



👩‍🔬, use only the information provided to you by the user to answer the user''s question. 
            If the information is insufficient for 👩‍🔬 to derive an answer, just say ''I cannot find relevant information in my document store to answer the question correctly.'' 
            👩‍🔬 is to provide an in-depth analysis to the question based only on the information provided by the user and nothing more.
            👩‍🔬 will give a real-life example to support illustrating your point and contrasting it with a counter-example. 
            👩‍🔬 will also proofread and edit the content before responding. 
            👩‍🔬 will provide your own reasoned subjective perspective, noting where your view differs from or expands on the contents.
            Rules:
                - Consolidate the materials provided by the user and then organise them point by point.
                - Provide as much details as you can extract from the materials provided by the user.
                - Begin your re


            Refer to the following context: `["well as salt-baked chicken. With the diversity presented in a single dish, Singapore's chicken rice definitely draws in curious diners to uncover more of the Lion City's chicken rice culture. Being equally fascinated with Singapore's chicken rice, the MICHELIN Guide inspectors share with us where to get the best ones in Singapore, so read on below! Boon Tong Kee (Balestier Road) 399, 401, 403 Balestier Road, 329801 Singapore $", "15 September 2023 There are many dishes that are very much representative of Singapore, and one of the most iconic of them is, of course, chicken rice. Loosely referred to as Singapore's national dish, the ubiquitous Hainanese chicken rice requires a meticulous preparation process. To make this classic chicken rice dish, the chicken is first scalded until cooked through before it is dipped in ice-cold water to keep the meat soft and tender. Served with chicken broth, ginger sauce, black soy sauce, and chilli, the

## State 6. PROCESS



In [10]:
# ACT
agent.next()

# ASSERT
TestAgentHelper.ShowAgent(agent)


step=6
state=PROCESS
tool_choice=none
tool_name='text




MONOLOGUE



👩‍🔬, use only the information provided to you by the user to answer the user''s question. 
            If the information is insufficient for 👩‍🔬 to derive an answer, just say ''I cannot find relevant information in my document store to answer the question correctly.'' 
            👩‍🔬 is to provide an in-depth analysis to the question based only on the information provided by the user and nothing more.
            👩‍🔬 will give a real-life example to support illustrating your point and contrasting it with a counter-example. 
            👩‍🔬 will also proofread and edit the content before responding. 
            👩‍🔬 will provide your own reasoned subjective perspective, noting where your view differs from or expands on the contents.
            Rules:
                - Consolidate the materials provided by the user and then organise them point by point.
                - Provide as much details as you can extract from the materials provided by the user.
                - Begin your re


            Refer to the following context: `["well as salt-baked chicken. With the diversity presented in a single dish, Singapore's chicken rice definitely draws in curious diners to uncover more of the Lion City's chicken rice culture. Being equally fascinated with Singapore's chicken rice, the MICHELIN Guide inspectors share with us where to get the best ones in Singapore, so read on below! Boon Tong Kee (Balestier Road) 399, 401, 403 Balestier Road, 329801 Singapore $", "15 September 2023 There are many dishes that are very much representative of Singapore, and one of the most iconic of them is, of course, chicken rice. Loosely referred to as Singapore's national dish, the ubiquitous Hainanese chicken rice requires a meticulous preparation process. To make this classic chicken rice dish, the chicken is first scalded until cooked through before it is dipped in ice-cold water to keep the meat soft and tender. Served with chicken broth, ginger sauce, black soy sauce, and chilli, the

 According to my document store, there is no information provided about any famous chicken rice in Singapore that also sells char siew. However, it is worth noting that the context does not mention any other dishes served by the restaurants mentioned. Therefore, it is possible that some of these restaurants may serve char siew as part of their menu, but this information is not provided in the context.

To answer your question more definitively, I would need additional information about which specific restaurants or chains of restaurants in Singapore are known for both chicken rice and char siew. Once I have that information, I can search my document store for more detailed information about those restaurants and their menus.

For example, let's say we know that the restaurant "Lau Pa Sat" is famous for both chicken rice and char siew. In that case, I could search my document store for more information about Lau Pa Sat and find out that they do indeed serve char siew as part of their me